In [ ]:
from src.MOF_builder.v2_builder import net_optimizer

import numpy as np


linker_cif = 'diedge.cif'
node_target_type = 'Zr'

#in database by calling MOF family name and node metal type, dummy node True or False
template_cif = 'fcu.cif' #from MOF family name in config dict file
node_cif = 'node2.cif' #from MOF family name  and node metal type in config dict file


fcu = net_optimizer()
fcu.analyze_template_ditopic(template_cif)
fcu.node_info(node_cif,node_target_type)
fcu.linker_info(linker_cif)
fcu.optimize()

 no symmetry operation
 no symmetry operation
P1 cell
P1 cell
P1 cell
P1 cell


 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          252     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.93767D+07    |proj g|=  2.23517D+01

At iterate    1    f=  6.93767D+07    |proj g|=  2.38419D+01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
  252      1     12      1     0     0   2.384D+01   6.938D+07
  F =   69376653.661462367     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
edge lengths: {('V0', 'V1'): 21.213, ('V1', 'V0'): 21.213, ('V0', 'V2'): 21.213, ('V2', 'V0'): 21.213, ('V0', 'V3'): 21.213, ('V3', 


   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.


In [2]:
supercell = [1,1,1]
#find unsaturated node idx
fcu.place_edge_in_net()
fcu.set_supercell(supercell)
fcu.make_supercell_ditopic()
fcu.make_eG_from_supereG_ditopic()
fcu.main_frag_eG()


In [3]:
fcu.make_supercell_range_cleaved_eG()
fcu.add_xoo_to_edge_ditopic()
fcu.find_unsaturated_node_eG()
fcu.set_node_terminamtion('methyl.pdb')
fcu.add_terminations_to_unsaturated_node()
fcu.remove_xoo_from_node()
fcu.write_node_edge_node_gro('fcu_ditopic')

#temp_save_eGterm_gro(fcu.eG,fcu.sc_unit_cell) #debug


  

fcu_ditopic .gro is saved
node_res_num:  14
edge_res_num:  72
term_res_num:  96


In [9]:
fcu.terms_eG


array([['1', 'TERM', 'X', ..., -3.150253303228628, 'C1', 11],
       ['1', 'TERM', 'C', ..., -4.219429736213236, 'C2', 11],
       ['1', 'TERM', 'Y', ..., -1.963327596541469, 'O1', 11],
       ...,
       ['96', 'TERM', 'H', ..., 43.15324920910757, 'H1', 49],
       ['96', 'TERM', 'H', ..., 44.22313877340248, 'H2', 49],
       ['96', 'TERM', 'H', ..., 44.222838890794534, 'H3', 49]],
      dtype=object)

# defects and postprocess of node residues split 
### todo: make defects on eG and add terminations 
### todo: add linker exchange defects by superimpose the x ;linker_termination can be realized by linker exchange;
### todo: split dummy nodes by use dummy dict info ;

In [21]:
def extract_node_name_from_gro_resindex(res_index, node_array):
    nodes_name = set()
    for node_ind in res_index:
        node_name = node_array[node_array[:,0]==str(node_ind)][:,-1]
        name_set = set(node_name)
        nodes_name = nodes_name.union(name_set)
    return nodes_name
extract_node_name_from_gro_resindex([1,2,3], fcu.nodes_eG)

{'V0_[0. 0. 0.]', 'V1_[0. 0. 0.]', 'V2_[0. 0. 0.]'}

# preprocess : fetch node, make dummy node, make linker
### todo: process node with pdb format, process_node_pdb: extract node and linker pdb info 

In [1]:
import os
import numpy as np
import veloxchem as vlx
import networkx as nx
from src.MOF_builder.add_dummy2node import nodepdb2G,add_dummy_atoms_nodepdb
from src.MOF_builder.frag_recognizer import process_linker_molecule
from src.MOF_builder.fetchfile import  fetch_pdbfile,read_mof_top_dict,find_data_folder,copy_file


#from node pdb make dummy node pdb
#todo: make process node to process node pdb

def nodepdb(filename):
        xyzlines = ''
        inputfile = str(filename)
        with open(inputfile, "r") as fp:
            content = fp.readlines()
            #linesnumber = len(content)
        data = []
        for line in content:
            line = line.strip()
            if len(line)>0: #skip blank line
                if line[0:6] == "ATOM" or line[0:6] == "HETATM":
                    value_atom = line[12:16].strip()  # atom_label
                    value_x = float(line[30:38])  # x
                    value_y = float(line[38:46])  # y
                    value_z = float(line[46:54])  # z
                    atom_type = line[67:80].strip() # atom_note
                    formatted_xyz_line = "{:6s}  {:8.3f} {:8.3f} {:8.3f}".format(atom_type, value_x, value_y, value_z)
                    xyzlines+=formatted_xyz_line+'\n'
                    data.append([value_atom,atom_type,value_x,value_y,value_z])
        head = str(len(data))+'\n'+'\n'
        xyz_lines = head + xyzlines
        return xyz_lines,np.vstack(data)




In [2]:
data_path = find_data_folder()
mof_top_dict = read_mof_top_dict(data_path)
#print mof_top_dict keys fit to screen
print('Available MOF Family:')
print(' '.join(mof_top_dict.keys()))


Available MOF Family:
TEST-MOF UiO-66 UiO-67 UiO-68 PCN-222 MOF-545 NU-1000 PCN-224 MOF-525 MOF-535 MOF-808 MOF-818 PCN-777 NU-901 MIL-100 MIL-101 PCN-228 PCN-229 PCN-230 MIL-53


In [3]:
mof_family = 'UiO-66'
node_connectivity = mof_top_dict[mof_family]['node_connectivity']
linker_topic = mof_top_dict[mof_family]['linker_topic']
template_cif = mof_top_dict[mof_family]['topology']+ '.cif'
DUMMY_NODE = True
metal = 'Zr'
linker_file = 'outndi.xyz'
#nodes_database_path = os.path.join(data_path,'nodes_database')
linkers_database_path = ''
nodes_database_path = os.path.join(data_path,"nodes_database")
node_pdb = fetch_pdbfile(nodes_database_path, [str(node_connectivity)+'c',metal],['dummy'])
#if node_termination == 'methyl' :
   # n_term_file = '../data/terminations_database/methyl.pdb'
for i in range(len(node_pdb)):
    node_pdb_database = os.path.join(data_path,'nodes_database/'+node_pdb[i])
    target_node_path = os.path.join('nodes',node_pdb[i])
    copy_file(node_pdb_database,target_node_path)

if DUMMY_NODE:
    #node_pdbfile = 'data/nodes_database/12c_Zr.pdb'
    #print(node_pdbfile)
    nodeG = nodepdb2G(target_node_path,metal)
    all_lines,dummy_ciffile = add_dummy_atoms_nodepdb(target_node_path,metal,nodeG)
if DUMMY_NODE:
    os.remove(target_node_path)
    print(target_node_path,'removed')

molecule = vlx.Molecule.read_xyz_file(linker_file)
center_frag_nodes_num,center_Xs,single_frag_nodes_num,frag_Xs= process_linker_molecule(molecule,linker_topic)

templates_dir = 'data/template_database' #default 
nodes_dir = os.path.dirname(target_node_path)#'nodes' #default 
edges_dir = 'edges' #default 

if DUMMY_NODE:
    keywords = [str(node_connectivity)+'c',metal,'dummy']
    nokeywords = []
else:
    keywords = [str(node_connectivity)+'c',metal]
    nokeywords = ['dummy']

dummy_node_name = fetch_pdbfile(nodes_dir,keywords, nokeywords)[0]


found the file including ['12c', 'Zr']
File copied from /Users/chenxili/GitHub/vlx_mofbuilder/data/nodes_database/12c_Zr.pdb to nodes/12c_Zr.pdb
nodes/12c_Zr_dummy_dict saved
nodes/12c_Zr_dummy.pdb saved
nodes/12c_Zr.pdb removed
center is two points
[9, 14]
ditopic linker: center are two points
find connected X in edge:   24
find connected X in edge:   39
center_frag: 46 [18, 33]
found the file including ['12c', 'Zr', 'dummy']
